In [1]:
#pip install pyathena

In [2]:
#Libraries
from pyathena import connect
import pandas as pd
import datetime as dt
import numpy as np

In [3]:
# DB Setting
bucket_name = 's3://viamericas-datalake-dev-us-east-1-283731589572-athena/'
origin_name = 'AwsDataCatalog'
database_name= 'analytics'
table_name = 'daily_check'

In [4]:
#Connection
conn = connect(s3_staging_dir=f'{bucket_name}queryresults/',
               region_name='us-east-1')
df = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table_name};', conn)

/tmp/ipykernel_1953/3429179766.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table_name};', conn)


In [5]:
df.head()

,payer,country,date,tx,amount
0,BANCO GANADERO (BOLIVIA),BOLIVIA,2023-04-13,0,0.00
1,CARIBE EXPRESS,DOMINICAN REPUBLIC,2023-01-23,1117,350050.40
2,GRUPO EXITO (TN),COLOMBIA,2023-01-23,247,52423.90
3,BANPAIS (RED CHAPINA),HONDURAS,2022-10-31,146,59000.50
4,TELECOMM TELEGRAFOS (APPRIZA),MEXICO,2022-05-15,1062,640446.24


In [6]:
# Convert the 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'])

In [7]:
# Grouping by 'payer' and 'country' concatenated for this level of granularity
df['payer_country'] = df['payer'] + '_' + df['country']

In [8]:
# Chequeo montos: Mismo valor que el dataset de la notebook Aging
df['amount'].sum()

25110624454.414906

In [9]:
def aging_filter(df):
    # Find the last date in the sample
    last_date_sample = df['date'].max()

    # Calculate the limit date, one day before the last date in the sample
    limit_date = last_date_sample - pd.Timedelta(days=1)

    # Find the first and last date per 'payer_country' and calculate total_amount and count
    result = (
        df.groupby('payer_country')
        .agg(
            first_date=('date', 'min'),
            last_date=('date', 'max'),
            total_amount=('amount', 'sum'),
            total_transactions=('tx', 'sum')  # Add this line for total_transactions
        )
        .reset_index()
    )

    # Calculate the 'age_payer' variable accurately using the calculated limit_date
    result['age_payer'] = ((limit_date - result['first_date']).dt.days / 30).round(2)

    # Calculate the 'active_payer' variable
    result['active_time'] = ((result['last_date'] - result['first_date']).dt.days / 30).round(2)

    # Calculate the 'inactive' variable
    result['inactive_time'] = ((limit_date - result['last_date']).dt.days / 30).round(2)

    # Sort the final DataFrame by 'total_amount' from highest to lowest
    result = result.sort_values(by='total_amount', ascending=False)

    # Filter the DataFrame based on conditions and return aging_universe
    aging_universe = result.loc[
        (result.age_payer >= 3) & 
        (result.inactive_time <= 3) & 
        (result.total_amount > 10000) & 
        (result.total_transactions > 50)
    ]

    return aging_universe

In [10]:
# Defining Universe
df = df[df['amount'] != 0] # Excluding 0 (flag A & Flag C), defined in EDA

df_aging = aging_filter(df) #Filtering 'payer_country' based on Aging notebook
df_filtered = df[df['payer_country'].isin(df_aging['payer_country'])] # Applying aging filters 

In [11]:
# Date format
df_filtered['date'] = pd.to_datetime(df_filtered['date']).dt.date

/tmp/ipykernel_1953/1252395900.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['date'] = pd.to_datetime(df_filtered['date']).dt.date


In [12]:
df_filtered['amount'].sum()

24309892564.971

In [13]:
##24309913777.781002 valor obtenido en aging, perdemos un payer y 18000 usd

In [14]:
df[df['country'] == 'MEXICO']['amount'].sum()

13655009099.1769

In [15]:
df_filtered[df_filtered['country'] == 'MEXICO']['amount'].sum()

12910028578.2625

In [16]:
df[df['country'] == 'MEXICO']['amount'].sum() - df_filtered[df_filtered['country'] == 'MEXICO']['amount'].sum()

744980520.9144001

In [17]:
df[df['payer_country'] == 'ELEKTRA (MEXICO)_MEXICO']['amount'].sum()

6397946743.728701

In [18]:
df_filtered[df_filtered['payer_country'] == 'ELEKTRA (MEXICO)_MEXICO']['amount'].sum()

6397946743.728701

In [19]:
df_filtered['country'].unique()

array(['DOMINICAN REPUBLIC', 'COLOMBIA', 'HONDURAS', 'MEXICO',
       'EL SALVADOR', 'GUATEMALA', 'BRAZIL', 'NICARAGUA', 'PERU', 'CHILE',
       'PHILIPPINES', 'UNITED STATES (ATM & DEPOSITS)', 'ECUADOR',
       'ARGENTINA', 'SIERRA LEONE', 'BOLIVIA', 'PORTUGAL', 'GAMBIA, THE',
       'HAITI', 'GHANA', 'SENEGAL', 'NIGERIA', 'UNITED STATES', 'INDIA',
       'GUINEA', 'INDONESIA', 'CAMEROON', 'JAMAICA', 'GERMANY',
       'THAILAND', 'LIBERIA', 'NEPAL', 'FRANCE', 'ITALY', 'BURKINA FASO',
       "COTE D'IVOIRE (IVORY COAST)", 'MALI', 'TOGO', 'BANGLADESH',
       'UNITED KINGDOM', 'SPAIN', 'BENIN', 'MALAYSIA', 'BELGIUM',
       'COSTA RICA'], dtype=object)

In [20]:
df_filtered.payer_country.nunique()

130

In [21]:
df_filtered.head()

,payer,country,date,tx,amount,payer_country
1,CARIBE EXPRESS,DOMINICAN REPUBLIC,2023-01-23,1117,350050.40,CARIBE EXPRESS_DOMINICAN REPUBLIC
2,GRUPO EXITO (TN),COLOMBIA,2023-01-23,247,52423.90,GRUPO EXITO (TN)_COLOMBIA
3,BANPAIS (RED CHAPINA),HONDURAS,2022-10-31,146,59000.50,BANPAIS (RED CHAPINA)_HONDURAS
4,TELECOMM TELEGRAFOS (APPRIZA),MEXICO,2022-05-15,1062,640446.24,TELECOMM TELEGRAFOS (APPRIZA)_MEXICO
5,CUSCA (CITI - RECIBA NETWORKS),EL SALVADOR,2023-02-06,868,280568.50,CUSCA (CITI - RECIBA NETWORKS)_EL SALVADOR


In [22]:
### RATES ###
df_rates=pd.read_csv('Rates_csv/rates_2021_2023.csv') # CHEQUEAR FUENTE DE ORIGEN

In [ ]:
# Selecting columns & renaming
df_rates = df_rates[['feed_date','symbol','feed_price']] 
df_rates = df_rates.rename(columns={'feed_date':'date'})

In [ ]:
df_rates['date'] = pd.to_datetime(df_rates['date']).dt.date

In [ ]:
# Function to keep last register, per day

def get_closing_prices(data, start_date, end_date):
    # Convert the 'FEED_DATE' column to datetime type if it is not already
    data['date'] = pd.to_datetime(data['date'])

    # Filter by date range
    data_filtered = data[(data['date'] >= start_date) & (data['date'] <= end_date)]

    # Sort by 'FEED_DATE' to ensure that the data is in chronological order
    data_filtered = data_filtered.sort_values(by='date')

    # Group by 'SYMBOL' and 'FEED_DATE', taking the last value of 'FEED_PRICE'
    data_grouped = data_filtered.groupby(['symbol', pd.Grouper(key='date', freq='D')])['feed_price'].last().reset_index()
    
#     # Generate evolution plots for each SYMBOL
#     for symbol in data_grouped['symbol'].unique():
#         symbol_data = data_grouped[data_grouped['symbol'] == symbol]

#         plt.figure(figsize=(10, 6))
#         plt.plot(symbol_data['date'], symbol_data['feed_price'], label=symbol)
#         plt.title(f'Closing prices evolution for {symbol}')
#         plt.xlabel('Date')
#         plt.ylabel('Closing Price')
#         plt.legend()
#         plt.show()
    
    return data_grouped

In [ ]:
start_date = '2021-01-01'
end_date = '2023-10-21'

rates = get_closing_prices(df_rates, start_date, end_date)

In [ ]:
rates

In [ ]:
rates.info()

In [ ]:
#Date format
rates['date'] = rates['date'].dt.date

In [ ]:
rates.head(4)

In [ ]:
###WE OBTAIN THE LAGS OF UP TO 7 DAYS FOR THE EXCHANGE RATE 
for i in range(1, 8):
        nombre_columna = f'feed_price_retardo_{i}'
        rates[nombre_columna] = rates['feed_price'].shift(i)

In [ ]:
rates['var_rate_lag1']=rates.feed_price_retardo_1-rates.feed_price_retardo_2
rates['var_rate_lag2']=rates.feed_price_retardo_2-rates.feed_price_retardo_3
rates['var_rate_lag3']=rates.feed_price_retardo_3-rates.feed_price_retardo_4
rates['var_rate_lag4']=rates.feed_price_retardo_4-rates.feed_price_retardo_5
rates['var_rate_lag5']=rates.feed_price_retardo_5-rates.feed_price_retardo_6
rates['var_rate_lag6']=rates.feed_price_retardo_6-rates.feed_price_retardo_7

In [ ]:
rates.head()

In [ ]:
# Filling NaN
rates=rates.fillna(method='bfill')
rates=rates.fillna(method='ffill')

In [ ]:
rates_dict = {
    'USDBRL': 'BRAZIL', # Bz Real 
    'USDINR': 'INDIA', # Indian Rupia
    'USDGTQ': 'GUATEMALA', #Quetzal 
    'USDMXN': 'MEXICO', #Mx Peso
    'USDPHP': 'PHILIPPINES' # Ph Peso
}

In [ ]:
rates['country'] = rates['symbol'].map(rates_dict)

In [ ]:
rates['country'].unique()

In [ ]:
rates.head()

In [ ]:
rates.shape

In [ ]:
# Combined df  containing only the rows that match both the 'date' and 'country' columns in both df (df_filtered and rates).
# WARNING!!! Includes only common columns [date, country] > Check rates_dict
df1 = pd.merge(df_filtered, rates[['date', 'country']], on=['date', 'country'], how='inner')

In [ ]:
df1['date'] = pd.to_datetime(df1['date']).dt.date

In [ ]:
df1['amount'].sum()

In [ ]:
### EFFECT OF CANCELED TRANSACTIONS ###

In [ ]:
table2_name = 'daily_sales_count_cancelled_v2'##WE LOAD THE BASE WITH CANCELLATIONS

df2 = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table2_name};', conn)

In [ ]:
df2.info()

In [ ]:
df2.amount.sum()

In [ ]:
df2['date'] = pd.to_datetime(df2['date']).dt.date

In [ ]:
for i in range(1, 8):
        nombre_columna = f'tx_cancelled_{i}'
        df2[nombre_columna] = df2['tx_cancelled'].shift(i)

In [ ]:
##ACÁ PUEDE HABER UN ERROR PORQUE HAY MAS PAYERS Y EL DATASET ME PARECE QUE NO ESTÁ ORDENADO POR PAYER COUNTRY DATE##
df_check=df2.copy()
df_check=df_check.loc[df_check.payer=='ELEKTRA (MEXICO)']

In [ ]:
df_check = df_check.sort_values(by='date')
df_check= df_check.set_index('date')

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.plot(df_check.index, df_check['tx_cancelled'], linestyle='-', color='b')
plt.title('Evolución de tx_cancelled a lo largo del tiempo')
plt.xlabel('Fecha')
plt.ylabel('tx_cancelled')
plt.grid(True)
plt.show()
##COMPARADO CON EL GRAFICO DE CANCELLATIONS SE VE IGUAL ASI QUE NO HAY PROBLEMA##

In [ ]:
df2.head()

In [ ]:
df2['var_tx_cancelled_1']=df2.tx_cancelled_1-df2.tx_cancelled_2
df2['var_tx_cancelled_2']=df2.tx_cancelled_2-df2.tx_cancelled_3
df2['var_tx_cancelled_3']=df2.tx_cancelled_3-df2.tx_cancelled_4
df2['var_tx_cancelled_4']=df2.tx_cancelled_4-df2.tx_cancelled_5
df2['var_tx_cancelled_5']=df2.tx_cancelled_5-df2.tx_cancelled_6
df2['var_tx_cancelled_6']=df2.tx_cancelled_6-df2.tx_cancelled_7

In [ ]:
df2

In [ ]:
df2.info()

In [ ]:
df2.amount.sum()

In [ ]:
df1.info()

In [ ]:
df1.amount.sum()

In [ ]:
##EL PROBLEMA ESTÁ EN DF1, ELEKTRA NO DEBERÍA DAR MENOS##
df_check2=df1[df1['payer_country'] == 'ELEKTRA (MEXICO)_MEXICO']
df_check2.amount.sum()

In [ ]:
df_final = pd.merge(df1,df2, on=['date','payer','country', 'amount'])

In [ ]:
df_final['date'] = pd.to_datetime(df_final['date'])

In [ ]:
df_final['amount'].sum()

In [ ]:
### DUMMIES ####

In [ ]:
# Weekday indicator (dummy)
df_final['weekday'] = (df_final['date'].dt.dayofweek < 4).astype(int)

In [ ]:
# Special dates list - Format: Month-day 
special_dates_list = [ '07-04', #US independence day 
                      '12-25', #Christmas
                      '05-11' # Mexico & Colombia - Mother's day
                     '01-01', #New Year
                     '01-02',
                     '12-26'] 

# Applying format process special_dates function
special_dates_list_processed = [(int(date.split('-')[0]), int(date.split('-')[1])) for date in special_dates_list]

In [ ]:
# To check special dates defined by list
def mark_special_dates(row):
    if (row['date'].month, row['date'].day) in special_dates_list_processed:
        return 1
    else:
        return 0

In [ ]:
# Marking special dates in df
df_final['special_dates'] = df_final.apply(lambda row: mark_special_dates(row), axis=1)

In [ ]:
df_final

In [ ]:
df_final['amount'].sum()

In [ ]:
df_elektra = df_final[df_final['payer_country'] == 'ELEKTRA (MEXICO)_MEXICO']

In [ ]:
df_elektra['amount'].sum()

In [ ]:
df_elektra.head()

In [ ]:
CHEQUEAR PORQUE A ELEKTRA LE CAE EL MONTO EN 1300 MILLONES, NO DEBERÍA PASAR